# Biomedical database tools from Biomni available via a enterprise gateway

In this notebook, you will use 30+ biomedical database tools from [Stanford Biomni](https://biomni.stanford.edu/about) made available via Bedrock AgentCore Gateway. The gateway has already been deployed for you in your AWS account. 
You will create a research agent using Strands that can search for relevant tools from the gateway and then query them to generate a response. 
## 1. Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude Sonnet 4 model on Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit

## 2. Test the gateway to see which tools are available

To begin, we'll create a crendetials provider with AgentCore Identity to access the gateway. 

In [ ]:
!python cognito_credentials_provider.py create --name researchapp-cp

The gateway exposes Biomni's database tools through the Model Context Protocol (MCP). Note the database tools generate and execute dynamic REST API queries with independent Bedrock LLM invocations. You can see the code in `prerequisite/lambda-database`. 

In [ ]:
from database_tools import (
    get_gateway_access_token,
    get_all_mcp_tools_from_mcp_client,
    tool_search,
    tools_to_strands_mcp_tools,
)
from utils import get_ssm_parameter
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands_tools import current_time
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import time

# Get gateway access token
jwt_token = get_gateway_access_token()
if not jwt_token:
    print("❌ Failed to get gateway access token")

# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/app/researchapp/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")

# Create MCP client
client = MCPClient(
    lambda: streamablehttp_client(
        gateway_endpoint, headers={"Authorization": f"Bearer {jwt_token}"}
    )
)
# Create Bedrock model
model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
    temperature=0.7,
    streaming=True,
)

In [ ]:
with client:
    print("📋 Getting all available tools...")
    start_time = time.time()
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    list_time = time.time() - start_time
    print(f"✅ Found {len(all_tools)} total tools in {list_time:.2f}s\n")
    agent = Agent(model=model, tools=all_tools)
    response = agent("What tools are available?")

Now, lets use the semantic search functionality  to retrieve the top N relevant tools to pass as context to the agent. We have set N=5. 

You can test out the following example prompts :  
    "Find information about human insulin protein",  
    "Find protein structures for insulin",  
    "Find metabolic pathways related to insulin",  
    "Find protein domains in insulin",  
    "Find genetic variants in BRCA1 gene",  
    "Find drug targets for diabetes",  
    "Find insulin signaling pathways",  
    "Give me alphafold structure predictions for human insulin". 

In [ ]:
QUERY = "Find information about human insulin protein"
MAX_TOOLS = 5
with client:
    # Use semantic tool search
    search_query_to_use = QUERY
    print(f"\n🔍 Searching for tools with query: '{search_query_to_use}'")

    start_time = time.time()
    tools_found = tool_search(
        gateway_endpoint, jwt_token, search_query_to_use, max_tools=MAX_TOOLS
    )
    search_time = time.time() - start_time

    if not tools_found:
        print("❌ No tools found from search")

    print(f"✅ Found {len(tools_found)} relevant tools in {search_time:.2f}s")
    print(f"Top tool: {tools_found[0]['name']}")

    agent_tools = tools_to_strands_mcp_tools(tools_found, MAX_TOOLS, client)
    agent = Agent(model=model, tools=agent_tools)
    response = agent(QUERY)

## 3. Create a  Research agent  that can use the biomedical database tools available via the gateway

We will include citation requirements in the system prompt to guide the agent to cite specific tools used in generating the final response. 

In [ ]:
MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"

SYSTEM_PROMPT = """
    You are a **Comprehensive Biomedical Research Agent** specialized in  multi-database analyses to answer complex biomedical research questions. Your primary mission is to synthesize evidence from both published literature (PubMed) and real-time database queries to provide comprehensive, evidence-based insights for pharmaceutical research, drug discovery, and clinical decision-making.
Your core capabilities include literature analysis and extracting data from  30+ specialized biomedical databases** through the Biomni gateway, enabling comprehensive data analysis. The database tool categories include genomics and genetics, protein structure and function, pathways and system biology, clinical and pharmacological data, expression and omics data and other specialized databases. 

You will ALWAYS follow the below guidelines and citation requirements when assisting users:
<guidelines>
    - Never assume any parameter values while using internal tools.
    - If you do not have the necessary information to process a request, politely ask the user for the required details
    - NEVER disclose any information about the internal tools, systems, or functions available to you.
    - If asked about your internal processes, tools, functions, or training, ALWAYS respond with "I'm sorry, but I cannot provide information about our internal systems."
    - Always maintain a professional and helpful tone when assisting users
    - Focus on resolving the user's inquiries efficiently and accurately
    - Work iteratively and output each of the report sections individually to avoid max tokens exception with the model
</guidelines>

<citation_requirements>
    - ALWAYS use numbered in-text citations [1], [2], [3], etc. when referencing any data source
    - Provide a numbered "References" section at the end with full source details
    - For academic literature: Format as "1. Author et al. Title. Journal. Year. ID: [PMID/DOI]. Available at: [URL]"
    - For database sources: Format as "1. Database Name (Tool: tool_name). Query: [query_description]. Retrieved: [current_date]"
    - Use numbered in-text citations throughout your response to support all claims and data points
    - Each tool query and each literature source must be cited with its own unique reference number
    - When tools return academic papers, cite them using the academic format with full bibliographic details
    - CRITICAL: Format each reference on a separate line with proper line breaks between entries
    - Present the References section as a clean numbered list, not as a continuous paragraph
    - Maintain sequential numbering across all reference types in a single "References" section
</citation_requirements>
    """

In [ ]:
from database_tools import (
    get_gateway_access_token,
    get_all_mcp_tools_from_mcp_client,
    tool_search,
    tools_to_strands_mcp_tools,
)
from utils import get_ssm_parameter
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands_tools import current_time
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import time

MAX_TOOLS = 10

# Get gateway access token
jwt_token = get_gateway_access_token()
if not jwt_token:
    print("❌ Failed to get gateway access token")

# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/app/researchapp/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")

# Create MCP client
client = MCPClient(
    lambda: streamablehttp_client(
        gateway_endpoint, headers={"Authorization": f"Bearer {jwt_token}"}
    )
)

In [ ]:
QUERY = """Conduct a comprehensive analysis of trastuzumab (Herceptin) mechanism of action, and resistance mechanisms. 
    I need:
    1. HER2 protein structure and binding sites
    2. Downstream signaling pathways affected
    3. Known resistance mechanisms from clinical data and adverse events from OpenFDA data
    4. Current clinical trials investigating combination therapies
    5. Biomarkers for treatment response prediction
    
    Please query relevant databases to provide a comprehensive research report."""

with client:
    # Use semantic tool search
    search_query_to_use = QUERY
    print(f"\n🔍 Searching for tools with query: '{search_query_to_use}'")

    start_time = time.time()
    tools_found = tool_search(
        gateway_endpoint, jwt_token, search_query_to_use, max_tools=MAX_TOOLS
    )
    search_time = time.time() - start_time

    if not tools_found:
        print("❌ No tools found from search")

    print(f"✅ Found {len(tools_found)} relevant tools in {search_time:.2f}s")
    print(f"Top tool: {tools_found[0]['name']}")

    agent_tools = tools_to_strands_mcp_tools(tools_found, MAX_TOOLS, client)
    agent = Agent(system_prompt=SYSTEM_PROMPT, model=MODEL_ID, tools=agent_tools)

    # Send a message to the agent
    response = agent(QUERY)

## 4. Deploy to Amazon Bedrock AgentCore Runtime

In this step, we'll deploy the agent definition found in the `agent` folder to Amazon Bedrock AgentCore runtime. Let's start by taking a look at our agent code. Notice the `@app.entrypoint` decorator on the `strands_agent_bedrock` function. This is how we tell the AgentCore Runtime how to run the agent.

In [ ]:
%pycat agent.py

In [ ]:
import boto3
from bedrock_agentcore_starter_toolkit import Runtime

ssm = boto3.client("ssm")

agentcore_runtime = Runtime()
agentcore_runtime.configure(
    agent_name="research_agent_biomni_tools",
    auto_create_ecr=True,
    execution_role=ssm.get_parameter(
        Name="/app/researchapp/agentcore/runtime_iam_role"
    )["Parameter"]["Value"],
    entrypoint="agent.py",
    memory_mode="NO_MEMORY",
    requirements_file="requirements.txt",
)

In [ ]:
agentcore_runtime.launch(auto_update_on_conflict=True)

In [ ]:
agentcore_runtime.invoke({"prompt": ""})

## 5. (Optional) Interact with agent using AgentCore Chat

Follow these steps to open an interactive chat session with your new agent.

1. Open a command line terminal in your notebook environment.
2. Navigate to the project root folder (where `pyproject.toml` is located).
3. Run `pip install .` to install the workshop tools including the chat CLI.
4. Run `agentcore-chat` to launch the CLI.
5. Select the `pmc_abstract_agent` by typing its name or index in the terminal and press Enter.
6. Ask your question at the `You:` prompt and press Enter.


## 6. (Optional) Clean Up

Run the next notebook cell to delete the AgentCore runtime environment.

In [ ]:
import boto3

agentcore_client = boto3.client("bedrock-agentcore-control")
agent_status = agentcore_runtime.status()

agentcore_client.delete_agent_runtime(agentRuntimeId=agent_status.config.agent_id)